In [1]:
import joblib
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import(
                                    StratifiedKFold,
                                    cross_validate,
                                    GridSearchCV
)
from sklearn.metrics import (
                            accuracy_score,
                            precision_score,
                            confusion_matrix
)

### 1. Load the data

In [2]:
X_train = np.load('artifacts/X_train.npz')['arr_0']
Y_train = np.load('artifacts/y_train.npz')['arr_0']
X_test = np.load('artifacts/X_test.npz')['arr_0']
Y_test = np.load('artifacts/y_test.npz')['arr_0']

### 2. Define Multi Models


In [3]:
lr_param_grid = {
                'max_iter' : [1000, 5000, 10000],
}

dt_param_grid = {
                'max_depth' : [8, 12, 16, 20],
                'criterion' : ["gini", "entropy", "log_loss"]
}

rf_param_grid = {
                'n_estimators' : [50, 100, 150, 200],
                'max_depth' : [8, 12, 16, 20],
                'criterion' : ["gini", "entropy", "log_loss"]
}

param_grids = {
            'Logistic Regression' : lr_param_grid,
            'Decision Tree' : dt_param_grid,
            'Random Forest' : rf_param_grid
}

### 3. Define Multi Models

In [4]:
models = {
           "Logistic Regression" : LogisticRegression(),
           "Decision Tree" : DecisionTreeClassifier(),
           "Random Forest" : RandomForestClassifier()
}


### 4. Configure K-Fold CV

In [5]:
cv = StratifiedKFold(
                    n_splits=6,
                    random_state=42,
                    shuffle=True
)

### 5. Multi Model Training

In [6]:
grid_search_results = {}
for model_name, model in models.items():
    print(f"\n----Tuning {model_name} ----")

    param_grid = param_grids[model_name]

    grid_search = GridSearchCV(
                                estimator=model,
                                param_grid=param_grid,
                                cv=cv, scoring='f1',
                                verbose=1, return_train_score=False
    )

    print(f"Fitting gridSearchCV for {model_name}")

    grid_search.fit(X_train, Y_train)

    grid_search_results[model_name] = grid_search

    print(f"{model_name} gridSearchCV completed ...")
    print(f"Best paramaters: {grid_search.best_params_}")
    print(f"Best CV Score: {grid_search.best_score_}")


----Tuning Logistic Regression ----
Fitting gridSearchCV for Logistic Regression
Fitting 6 folds for each of 3 candidates, totalling 18 fits
Logistic Regression gridSearchCV completed ...
Best paramaters: {'max_iter': 1000}
Best CV Score: 0.7318169755021501

----Tuning Decision Tree ----
Fitting gridSearchCV for Decision Tree
Fitting 6 folds for each of 12 candidates, totalling 72 fits
Decision Tree gridSearchCV completed ...
Best paramaters: {'criterion': 'log_loss', 'max_depth': 20}
Best CV Score: 0.8358588129518206

----Tuning Random Forest ----
Fitting gridSearchCV for Random Forest
Fitting 6 folds for each of 48 candidates, totalling 288 fits
Random Forest gridSearchCV completed ...
Best paramaters: {'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 100}
Best CV Score: 0.8935184926885612
